# INTEGRATED INFORMATION THEORY (IIT) 3.0 
# Non-binary systems 

by [Juan Gomez](https://github.com/juanogo)
/ [Original Paper](https://journals.plos.org/ploscompbiol/article/file?id=10.1371/journal.pcbi.1003588&type=printable) / [Videos on YouTube](https://www.youtube.com/watch?v=i3G-Wt5az30)

## Introduction

[PyPhi](http://integratedinformationtheory.org/calculate.html) is a software tool that calculates IIT in small binary systems but does not allow for non-binary systems calculations. This implementation is intended as an add-on to the original PyPhi to compensate for such lack.

In terms of knowledge gain, it would be of great interest to Tononi's lab, to see how the number of concepts, and phi values (small phi and Big Phi) compare between systems with similar numbers of states but different numbers of states per element.

## Imports

In [199]:
import pandas as pd
import numpy as np
from scipy.stats import wasserstein_distance
from scipy.stats import entropy
import matplotlib.pyplot as plt
from itertools import combinations 
from itertools import product
import string

## A non-binary system of three elements (candidate system1): 
![title](non-binary.png)


In [200]:
def calculate_state_system1(a_prev,b_prev,c_prev, connectivity=False):
    """
    Calculates a new state of the candidate system1 described in the graph above:
    This function defines the behavior of the elements and the interaction of the whole system1.
    Given the previous state of the elements in the system1 (a_prev,b_prev,c_prev),
    it calculates and returns the new state of the system1's elements(a_next,b_next,c_next).
    Note: this function has to be re-defined every time the system changes. 
    """ 
    connections = {'A': ['B','C'], 'B': ['A','C'], 'C': ['A','B']}   
    if connectivity:
        return connections
    
    a_next= b_prev + c_prev
    if a_next > 0:
        a_next=1
    elif a_next< 0:
        a_next=-1
        
    b_next= a_prev + c_prev
    if b_next > 0:
        b_next=1
    elif b_next< 0:
        b_next=-1
        
    c_next= a_prev + b_prev
    if c_next > 0:
        c_next=1
    elif c_next< 0:
        c_next=-1
        
    return a_next, b_next, c_next

def xor(x,y):
    return 0 if ((x==0 and y==0) or (x==1 and y==1)) else  1

def calculate_state_system2(a_prev,b_prev,c_prev, connectivity=False):
    """
    This is a regular binary system (described in the paper) just for testing the code.
    """ 
    connections = {'A': ['B','C'], 'B': ['A','C'], 'C': ['A','B']}
    if connectivity:
        return connections
    
    a_next= b_prev or c_prev
    b_next= a_prev and c_prev
    c_next= xor(a_prev,b_prev)
    return a_next, b_next, c_next



In [201]:
def perturbate(system,prev_states, connectivity=False):
    """
    This function perturbates a system 'system', given the set of system's elements previous states 'prev_states'.
    Then, it returns the next states of the system 'system'.
    For each new candidate system this function needs to expand its options: system==1, system==2, etc.
    Also, for each option "system==n" added, a function "calculate_state_system n" must be created above.
    Example: perturbate(1,[-1,0,1]) perturbates system1 (system==1), in state [-1,0,1]
    """ 
   
    if system==1:
        return calculate_state_system1(prev_states[0],prev_states[1],prev_states[2] , connectivity)
    elif system==2:
        return calculate_state_system2(prev_states[0],prev_states[1],prev_states[2], connectivity)
    else:
        print("System not recognized ")


## TPM (Transition Probability Matrix) calculation:

A matrix that specifies the probability with which any state of a system transitions to any other system state. The TPM is determined by the mechanisms of a system and obtained by perturbing the system into all its possible states.

In [202]:
def tpm(system,number_nodes,states):
    """
    This function returns the TPM of a system "system", given its number of nodes and the possible states
    that a node in the system can take.
    Example: tpm(1,3,[-1,0,1]) returns the TPM of the system=1 (described above), whose 3 nodes can have each, one of
    three states: -1, 0 or 1
    """ 
    
    number_states=len(states)**number_nodes
    matrix=np.zeros((number_states,number_states))
    
    all_pos_states=list(product(states, repeat=number_nodes))
    all_pos_states=list(map(list, list(zip(*all_pos_states))[::-1]))

    #goes through all possible states and perturbates the system with all of them, one by one 
    for i in range(number_states):
        #next_state is the next state obtained by perturbating the system with the i-th state 
        next_state=list(perturbate(system,[row[i] for row in all_pos_states]))
        #goes through all possible states to relate them (0 or 1) with the one obtained by the perturbation
        for j in range(number_states):
            state=[row[j] for row in all_pos_states]
            if next_state==state:
                matrix[i][j]=1 #100% probabilty of this being the state obtained by the perturbation
            else:
                matrix[i][j]=0
        
    index = pd.MultiIndex.from_arrays(all_pos_states, names=list(string.ascii_uppercase[:number_nodes]))
    columns = pd.MultiIndex.from_arrays(all_pos_states, names=list(string.ascii_uppercase[:number_nodes]))
    TPM = pd.DataFrame(matrix,columns=columns, index=index)
    
    return TPM 


## Cause Repertoire calculation:

The probability distribution of potential past states of a system as constrained by a mechanism in its current state.

In [203]:
def cause_repertoire(mechanism,purview, state, tpm):
    """
    Given the 'tpm' of a system, this function returns the cause repertoire of a 'mechanism' over a 'purview', 
    in a certian system's 'state'. Note: Comments on the next function (effect_repertoire()) may apply to this one too.
    Example: TPM=tpm(2,3,[0,1]), mechanism=['B'], purview=['A'], state={'A':1,'B':0,'C':0 },
    cause_repertoire(mechanism,purview, state, TPM).
    """ 
    
    factor=np.power(tpm.shape[0],1/len(state))**(len(tpm.index.names)-len(purview))  
    #how long the prob. will be given the number of purview's elemnts
    factor2=np.power(np.power(tpm.shape[0],1/len(state)), len(purview))
    
    #group in revrse [::-1] just to preserve paper's notation
    #i.e. the fisrt variable varies faster [A, B] A: 0101 B:0011(slower)
    tpm=(tpm.groupby(purview[::-1]).sum())*(1/factor)
    
    if len(mechanism)>0:
        tpm=(tpm.transpose().groupby(mechanism).sum()).transpose()
        col=[state[i] for i in mechanism]
        return list(tpm.loc[:, tuple(col)]/sum(list(tpm.loc[:, tuple(col)]))) if len(mechanism)>1\
                                                        else list(tpm.loc[:, col[0]]/sum(list(tpm.loc[:, col[0]])))
    else:
        #the unconstraind is just the normal distribution (equal probability for all)
        return [1/factor2]*int(factor2)

## Effect Repertoire calculation:

The probability distribution of potential future states of a system as constrained by a mechanism in its current state.

In [204]:
def effect_repertoire(mechanism,purview, state, tpm):
    """
    Given the 'tpm' of a system, this function returns the effect repertoire of a 'mechanism' over a 'purview', 
    in a certian system's 'state'. The purview needs to have one element only (a product is needed otherwise).
    Example: TPM=tpm(2,3,[0,1]), mechanism=['B'], purview=['A'], state={'A':1,'B':0,'C':0 },
    effect_repertoire(mechanism,purview, state, TPM).
    """ 

    #if n variables are marginalized from the mechanism, it implies that 'number of states' rows 
    #have to be added n times. Therefore, the result has to be divided by 1/('number of states'^n)= 1/factor.
    #np.power(tpm.shape[0],1/len(state))=number of states | (len(tpm.index.names)-len(mechanism))= n
    factor=np.power(tpm.shape[0],1/len(state))**(len(tpm.index.names)-len(mechanism))  
    
    #marginalize purview
    tpm=(tpm.transpose().groupby(purview).sum()).transpose()    

    # marginalize set of mechanisms
    if len(mechanism)>0:
        tpm=(tpm.groupby(mechanism).sum())*(1/factor)
        row=[state[i] for i in mechanism]
        return list(tpm.loc[tuple(row), :]) if len(mechanism)>1  else list(tpm.loc[row[0], :])
    else:
        return list(tpm.sum(axis=0)/tpm.shape[0])

## Calculation of the Tensor Product of two Probability Distributions.
[Math details](https://ncatlab.org/nlab/show/tensor+product+of+distributions)

In [205]:
def tensor_product(a,b,num_states=0):
    """
    It takes two lists a and b and outputs their tensor product as a list too.  
    """ 
    #transform the lists into dataframe for processing
    a=pd.DataFrame(a).transpose()
    b=pd.DataFrame(b).transpose()
        
    #calculate the tensor product
    matrix_prod=np.tensordot(a,b,axes=[0,0])
    #reshape the result into a one dimensional array (list)
    product=np.reshape(np.transpose(matrix_prod), matrix_prod.shape[0]*matrix_prod.shape[1]).tolist()
    
    if num_states>0 and len(product)>num_states:
        return [np.sum(product[i:i+num_states]) for i in range(0,len(product),num_states)]
    else:        
        return product
    
    #return product

## Calculation of the Distance two Probability Distributions.
This measurment quantifies how much two distributions differ by taking into account the distance between system states. 
Two main measurments can be used: Earth mover’s distance (EMD) or Kullback–Leibler divergence (KLD)

In [206]:
def distance(a,b,dist_type):
    """
    Given the type of distance 'dist_type' this function returns the distance betwwen two prob. distributions.
    Note: for kld, the distributions need to have the same length, not being the case for emd.
    """ 
    if dist_type=='emd':
        return wasserstein_distance(a,b)*4
    elif dist_type=='kld':
        return entropy(a,b)
    else:
        print('Probability distribution distance not recognized')
    

## Get the repertoires (cause or effect) of a mechanism over a purview.


In [207]:
def get_repertoire(mechanism, purview, state, tpm, rep_type, full_sys=True):
    """
    This function gets a repertoire 'rep_type'(cause or effect) of the 'mechanism' over the 'purview'
    in a system with 'state' given its 'tpm'. The variable 'full_sys' indicates whether or not,
    the resultant probability is expressed over the purview elements only, or over the entire state space.  
    """
    
    if not purview: return [1]
    L=list()
    state_space=list(string.ascii_uppercase[:len(state)])
       
    if rep_type=='effect':
        for element in state_space:
            if element in purview:
                L.append(effect_repertoire(mechanism,[element], state, tpm))
            elif full_sys: #calculate the unconstrained prob. over this non-purview element 
                L.append(effect_repertoire([],[element], state, tpm))
    elif rep_type=='cause':
        
        if full_sys and len(purview)<len(state_space):
            return tensor_product(cause_repertoire(mechanism,purview, state, tpm) , cause_repertoire([],list(set(state_space)-set(purview)), state, tpm) )             
        return cause_repertoire(mechanism,purview, state, tpm)
    else:
        print ('type of repertoire not recognized (cause|effect)')
    
    result=L[0]
    if len(L)>1: #the purvie had more than one element and tensor product has to be performed
        for i in range(len(L)-1):
            result=tensor_product(result,L[i+1])  
            
    return result
    

## Calculate causal information (ci), effect information (ei) and causal-effect inf (cei) 


In [208]:
def get_information(a,b,inf_type, dist_type='emd'):
    
    if inf_type=='ci' or inf_type=='ei':
        return distance(a,b,dist_type)
    elif inf_type=='cei':
        return min(a,b)
    else:
        print('Type of information not recognized')  
    

## Partitions 
Given a mechanism-purview pair, it is necesary to partition it into parts in order to calculate its integrated information.

In [209]:
def partitions(mechanism,purview,show=False):
    '''
    This function takes the full mechanism and purview and returns all its partitions.
    The return has the following structure:
    mec_part= [[left_side_partiton1_mec, left_side_partiton2_mec,...] , [right_side_partiton1_mec, right_side_partiton2_mec,...]]
    mec_part= [[left_side_partiton1_pur, left_side_partiton2_pur,...] , [right_side_partiton1_pur, right_side_partiton2_pur,...]]
    The N-th partition has the following form:  
    (left_side_partitonN_mec/left_side_partitonN_pur) x (right_side_partitonN_mec/right_side_partitonN_pur)
    '''
         
    mec_left=list()    
    for i in  range(len(mechanism)+1):
        mec_left=mec_left+list(combinations(mechanism, i))        
    mec_left=[sorted(e) for e in mec_left]    
    mec_right=[ sorted(set(mechanism)-set(e)) for e in mec_left]
    
    pur_left=[[]]+purview
    pur_right=[sorted(set(purview)-set(e)) for e in pur_left]    
 
    if show: #print the partitions

        for i in range(len(mec_left)):
            for j in range(len(pur_left)):
                if not(i==0 and j==0):
                    print('(', ''.join(mec_left[i]),'/', ''.join(pur_left[j]),') X (', ''.join(mec_right[i]),'/',''.join(pur_right[j]),')')
    
    mec_part=[[],[]]
    pur_part=[[],[]]
    for i in range(len(mec_left)):
            for j in range(len(pur_left)):
                if (mec_left[i]!=[] or pur_left[j]!=[]) and (mec_right[i]!=[] or pur_right[j]!=[]) :
                    mec_part[0].append(list(mec_left[i]))
                    mec_part[1].append(list(mec_right[i]))
                    pur_part[0].append(list(pur_left[j]))
                    pur_part[1].append(list(pur_right[j]))
            
 
    return mec_part, pur_part
    

## Minimum information partition and φ “small-phi”
φ ‘‘small phi’’ is calculated as the distance between two probability distributions: the cause-effect repertoire specified by the whole mechanism is compared against the causeeffect repertoire of the partitioned mechanism. Of the many possible ways to partition a mechanism, integrated information is evaluated across the minimum information partition (MIP), the partition that makes the least difference to the cause and effect repertoires (in other words, the minimum ‘‘difference’’ partition).


In [210]:
def mip(mechanism, purview, mec_part, pur_part,state,TPM, direction, show=False):
    '''
    This function takes the full mechanism and purview and all its partitions (mec_partitioned, pur_partitioned,).
    Then, it finds the MIP for both cause or effect (according to 'direction')  and the small phi (φ) of each.    
    '''
    num_states=TPM.shape[0] 
    
    mec_rep=get_repertoire(mechanism,purview, state,TPM,direction,True)
    small_phi=float('Inf')
    mip=[[],[],[],[]]
    for i in range (len(mec_part[0])):
        part_mec_rep=tensor_product( get_repertoire(mec_part[0][i],pur_part[0][i], state,TPM,direction,True),\
                        get_repertoire(mec_part[1][i],pur_part[1][i], state,TPM,direction,True),num_states) 
        dist=distance(mec_rep,part_mec_rep,'emd')
        if show: print(mec_part[0][i], pur_part[0][i],'X',mec_part[1][i],pur_part[1][i], dist)
        if dist<small_phi:
            small_phi=dist
            mip=[mec_part[0][i], mec_part[1][i],pur_part[0][i],pur_part[1][i]]

    if show: print('MIP',direction,'= ',mip[0],mip[2],'X',mip[1],mip[3],'-  MIPφ',direction,'=',small_phi )
        
    return mip, small_phi

## MICE (maximally irreducible cause-effect repertoire):
The cause-effect repertoire of a concept, i.e., the cause-effect
repertoire that generates a maximum of integrated information Q among all possible purviews.

In [211]:
def mice(mechanism,purview,state,TPM,direction,show=False ):
    '''
    This function returns the Minimally irreducible cause - efefct.
    Dependingo on the direction, it calculates either the cause or the effect. 
    The purview, its partition and small phi, whose small phi was max among all purviews.
    '''
    
    power_set=list()
    mi_small_phi=0  
    mi_rep=list()
    
    for i in  range(1,len(purview)+1):
            power_set=power_set+sorted(combinations(purview, i))     

    for possible_purview in power_set:
        part_mec, part_poss_pur=partitions(mechanism,list(possible_purview),show=False)
        if show: print('Mechanism:',mechanism,'possible purview:',list(possible_purview))
        mip_crr, small_phi=\
        mip(mechanism, list(possible_purview), part_mec, part_poss_pur,state,TPM, direction,show=False)
        if show: print('MIP',direction,'= ',mip_crr[0],mip_crr[2],'X',mip_crr[1],mip_crr[3],'-  MIPφ',direction,'=',small_phi,'\r\n')
        if small_phi>=mi_small_phi:
            mi_small_phi=small_phi
            mi_rep=mip_crr
            final_pur=list(possible_purview)
    if show: print('>>Maximally irreducible', direction,'= pueview:',final_pur,'mip:',mi_rep[0],mi_rep[2],'X',mi_rep[1],mi_rep[3],mi_small_phi,'\r\n')
    return final_pur, mi_small_phi

## Concept: 
A set of elements within a system and the
maximally irreducible cause-effect repertoire it specifies, with
its associated value of integrated information QMax. The
concept expresses the causal role of a mechanism within a
complex

In [212]:
def concept(mechanism,purview,state,TPM):  
    '''
    This function simples requests the mice for both cause and effect of a mechanism
    Then it renders then and returns them as the concept of the mechanism
    '''
    
    core_pur_cause, phi_cause=mice(mechanism,purview,state,TPM,'cause',show=False )
    core_pur_effect, phi_effect=mice(mechanism,purview,state,TPM,'effect',show=False )
    
    print('The concept of the mechanism', mechanism, 'is:')
    print('cause:',mechanism,'/', core_pur_cause, '-', phi_cause)
    print('effect:',mechanism,'/', core_pur_effect, '-', phi_effect, '\r\n')
    
    return get_repertoire(mechanism, core_pur_cause,state,TPM,'cause',True),\
            get_repertoire(mechanism, core_pur_effect,state,TPM,'effect',True), min(phi_cause, phi_effect)  

In [213]:
def tpm_cut(system_num,TPM,system,sub_sys1,sub_sys2, show=False):
    ''''
    This functions cuts a system and recalculates the TPM for the resulting cut system
    It does excactltly as shown in the technical report S2:
    finds the individual TPMs for each element (after the cut) and it expans thos TPM,
    finally it multiplies them to combine then in the TPM to be returned 
    '''
    
    connections = perturbate(system_num,[0,0,0], True) #returns the connectivity map of the system
    list_connections=list()
    list_elem_tpm=list()
    num_states=np.power(TPM.shape[0],1/len(system))

    for element in system: #this creates the tpm for each element
        for connection in connections: #tpm contains only the other elements connected to element (as inputs)
            if connection in connections[element] and connection not in sub_sys1:#sub_sys1 loses its causality (cut)
                list_connections.append(connection)
        if list_connections==system: # if the elment is connected to everyone (including itself), no marg. of any input
            list_elem_tpm.append(TPM.transpose().groupby(element).sum().transpose())
        else:
            if len(list_connections)==0:#element got disconnected, so it depends on itself 
                list_connections=[element]
            list_elem_tpm.append((TPM.transpose().groupby(element).sum().transpose().groupby(list_connections[::-1]).sum())*\
            (1/(num_states* len(set(system)-set(list_connections)))))
            if len(list_connections)==1 and list_connections[0]==element: #if it dpends on itself 
                dummy=np.zeros((int(num_states), int(num_states)), int)
                np.fill_diagonal(dummy, 1)
                list_elem_tpm[-1]=pd.DataFrame(dummy,columns=list_elem_tpm[-1].columns, index=list_elem_tpm[-1].index)               
        if show: print('TPM for element ', element, ': \r\n',list_elem_tpm[-1],'\r\n' )  
        list_connections=list()#empty for the next element
        
    
    
    list_elem_tpm_exp=list()
    for df in list_elem_tpm:#now we expand each element tpm 
        a=df.values.tolist()
        b=list()
        for i in range(len(a)):
            b=b+[a[i][:]]* int(TPM.shape[0]/df.shape[0])
        list_elem_tpm_exp.append(pd.DataFrame(np.asarray(b),columns=df.columns, index=TPM.index))
                
    
    temp=list_elem_tpm_exp[0]#multiply all the element's tpm to obtain a new tpm for the cut system 
    for i in range(1,len(list_elem_tpm_exp)):
        temp=multi_tpms(temp,list_elem_tpm_exp[i])
        
                
    return pd.DataFrame(temp.values,columns=TPM.columns, index=TPM.index)
    
#tpm_cut(2,TPM,['A','B'],['A'],['B'],show=True)

In [214]:
def multi_tpms(a,b):
    '''
    Multiplies two TPM matrixes
    '''

    a1=np.array(a.values.tolist()).transpose()
    a1=a1.reshape(a1.shape[0],a1.shape[1],1)
    a2=np.array(b.values.tolist())
    
    ans=np.array([i.transpose()  for i in a1*a2]).transpose().reshape(a.shape[0],a.shape[1]*b.shape[1])
    
    return pd.DataFrame(ans)

## Conceptual structure, constellation of concepts (C): A
conceptual structure is the set of all concepts specified by a
candidate set with their respective QMax values, which can be
plotted as a constellation in concept space.

In [215]:
def ces(mechanism,purview,state,TPM):
    ''''
    this functions calculates the cause and effect structure of a system 
    All its concepts but those with 0
    '''
    
    mec_set=list()
  
    for i in  range(1,len(mechanism)+1):
            mec_set=mec_set+sorted(combinations(mechanism, i))   
    
    structure=list()
    for possible_mec in mec_set:
        con_cause, con_effect, small_phi=concept(list(possible_mec),purview,state,TPM)
        if small_phi==0:
            con_cause=get_repertoire([],purview,state,TPM,'cause',True)
            con_effect=get_repertoire([],purview,state,TPM,'effect',True)
        structure.append([con_cause, con_effect, small_phi])
    
    return structure 
    
#ans=ces(mechanism,purview,state,TPM)

In [216]:
def calculate_phi(ce_struct1, ce_struct2):
    '''
    This function compares two cause and effect structures
    The first, comes from the entire system the other from a cut system
    '''
    
    phi=0
    for concept1, concept2 in zip(ce_struct1, ce_struct2):
        phi=phi+(distance(concept1[0],concept2[0],'emd')+distance(concept1[1],concept2[1],'emd'))*concept1[2] 
    
    print ('Φ =:',phi, '\r\n')
    return phi 
    
    
#calculate_phi(ans, ans)    

In [217]:
def iit(system_num, elements_name, TPM,state, possible_states ):
    '''
    This function calculates PHI of all possibles subsystems (or candidate systems)
    if a concept in one ce structure is 0, it uses the unconstrained distributions for comparison
    '''
    
    #TPM=tpm(system_num,len(elements_name),possible_states)
    whole_struct=ces(elements_name,elements_name,state,TPM)
    
    sub_sys=list()
    for i in  range(1,len(elements_name)):
        sub_sys=sub_sys+sorted(combinations(elements_name, i)) 
        
    big_phi=float('Inf')
    
    for sub_sys1 in sub_sys:
        print('CUT : ',list(sub_sys1), sorted(set(elements_name) - set(sub_sys1)) , '\r\n')
        TPM=tpm_cut(system_num, TPM, elements_name, list(sub_sys1), sorted(set(elements_name) - set(sub_sys1))  ) 
        cut_struct=ces(elements_name,elements_name,state,TPM)
        phi_cut=calculate_phi(whole_struct, cut_struct)
        if phi_cut< big_phi:
            big_phi=phi_cut
            
    print('Φ = ', big_phi)
    
        

#iit(2, ['A','B','C'], {'A':1,'B':0,'C':0 }, [0,1])

## EXAMPLES (cause-effect repertoire calculations).


### Example 1:
Let's reproduce the results for a binary system as shown in the original paper [page 7](https://journals.plos.org/ploscompbiol/article/file?id=10.1371/journal.pcbi.1003588&type=printable).

![title](paper.png)


In [218]:
#define the TPM for a system '2' (see description at the very top) with three binay elemnts
TPM=tpm(2,3,[0,1])

Cause repertoire:

In [219]:
mechanism=['A'] 
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
ans=get_repertoire(mechanism,purview, state,TPM, 'cause',True)
list(np.around(np.array(ans),3))

[0.0, 0.0, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167]

Unconstrained past:

In [220]:
mechanism=[] 
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
get_repertoire(mechanism,purview, state,TPM, 'cause',True)

[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]

Effect repertoire:

In [221]:
mechanism=['A'] 
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
get_repertoire(mechanism,purview, state,TPM, 'effect',True)

[0.0625, 0.1875, 0.0625, 0.1875, 0.0625, 0.1875, 0.0625, 0.1875]

Unconstrained future:

In [222]:
mechanism=[] 
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
get_repertoire(mechanism,purview,state,TPM, 'effect',True)

[0.09375, 0.28125, 0.03125, 0.09375, 0.09375, 0.28125, 0.03125, 0.09375]

### Example 2:
Let's calculate some cause and effect repertoires for the non-binary system of three elements, 
whose description and graph are shown on the very top of this Jupyter Notebook:


In [223]:
TPM=tpm(1,3,[-1,0,1])
TPM

,,A,-1,0,1,-1,0,1,-1,0,1,-1,...,1,-1,0,1,-1,0,1,-1,0,1
,,B,-1,-1,-1,0,0,0,1,1,1,-1,...,1,-1,-1,-1,0,0,0,1,1,1
,,C,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,...,0,1,1,1,1,1,1,1,1,1
A,B,C,,,,,,,,,,,,,,,,,,,,,
-1,-1,-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,-1,-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-1,0,-1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
-1,1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Cause repertoire of A/ABC

In [224]:
mechanism=['A'] 
purview=['A','B','C']
state={'A':-1,'B':0,'C':1 }
ans=get_repertoire(mechanism,purview, state,TPM, 'cause',True)
ans=list(np.around(np.array(ans),1))
print(*ans) 

0.1 0.1 0.1 0.1 0.1 0.1 0.0 0.0 0.0 0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0


Cause repertoire of ABC/ABC

In [225]:
mechanism=['A','B','C'] 
purview=['A','B','C']
state={'A':-1,'B':0,'C':1 }
ans=get_repertoire(mechanism,purview, state,TPM, 'cause',True)
ans=list(np.around(np.array(ans),1))
print(*ans) 

0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0


Cause repertoire of []/ABC

In [226]:
mechanism=[] 
purview=['A','B','C']
state={'A':-1,'B':0,'C':1 }
ans=get_repertoire(mechanism,purview, state,TPM, 'cause',True)
ans=list(np.around(np.array(ans),3))
print(*ans) 

0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037


Effect repertoire of A/AB

In [227]:
mechanism=['A'] 
purview=['A','B']
state={'A':-1,'B':0,'C':1 }
ans=get_repertoire(mechanism,purview, state,TPM, 'effect',True)
ans=list(np.around(np.array(ans),3))
print(*ans) 

0.074 0.074 0.074 0.037 0.037 0.037 0.0 0.0 0.0 0.074 0.074 0.074 0.037 0.037 0.037 0.0 0.0 0.0 0.074 0.074 0.074 0.037 0.037 0.037 0.0 0.0 0.0


In [228]:
mechanism=['A'] 
purview=['A','B']
state={'A':-1,'B':0,'C':1 }
ans=get_repertoire(mechanism,purview, state,TPM, 'effect',False)
ans=list(np.around(np.array(ans),3))
print(*ans) 

0.222 0.222 0.222 0.111 0.111 0.111 0.0 0.0 0.0


## ... end of examples (cause-effect repertoire calculations).

## EXAMPLES (integrated information  / MPI and φ “small-phi”).


### Example 1:
Let's reproduce the results of the paper for MIP cause and MIP effect [page 9](https://journals.plos.org/ploscompbiol/article/file?id=10.1371/journal.pcbi.1003588&type=printable).

![title](fig1.png)

In [229]:
TPM=tpm(2,3,[0,1])
state={'A':1,'B':0,'C':0 }

In [230]:
a=get_repertoire(['A','B'],['C'], state,TPM,'cause',True)
b=get_repertoire(['C'],['A','B'], state,TPM,'cause',True)
ten=tensor_product(a,b,8)
print('AB/C X C/AB=',ten, '...Same as in the paper!!!')
cause_rep=get_repertoire(['A','B','C'],['A','B','C'], state,TPM,'cause',True)
print('ABC/ABC=',cause_rep, '...Same as in the paper!!!')
print(distance(cause_rep,ten,'emd'),"...The EMD used in the paper differs from Python's (not my fault)")

AB/C X C/AB= [0.25, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.25] ...Same as in the paper!!!
ABC/ABC= [0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0] ...Same as in the paper!!!
0.5 ...The EMD used in the paper differs from Python's (not my fault)


In [231]:
a=get_repertoire([],['B'], state,TPM,'effect',True)
b=get_repertoire(['A','B','C'],['A','C'], state,TPM,'effect',True)
ten=tensor_product(a,b,8)
print('ABC/AC X []/B=',ten, '...Same as in the paper!!!')
effect_rep=get_repertoire(['A','B','C'],['A','B','C'], state,TPM,'effect',True)
print('ABC/ABC=',effect_rep, '...Same as in the paper!!!')
print(distance(effect_rep,ten,'emd'),"...The EMD used in the paper differs from Python's (not my fault)")

ABC/AC X []/B= [0.0, 0.0, 0.0, 0.0, 0.75, 0.0, 0.25, 0.0] ...Same as in the paper!!!
ABC/ABC= [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0] ...Same as in the paper!!!
0.25 ...The EMD used in the paper differs from Python's (not my fault)


### Note:
Notice tha this implementation is flawless. The only "problem" is that the EMD used in the paper is different to the one implemented by Python (scipy.stats) named wasserstein_distance. This affects will affect the results (if compared to the papar's) from now on. 

### Example 2:
Let's reproduce the results of the tecnical report, in terms of calculating partitions:

![title](fig2.png)

In [232]:
mechanism=['A','C']
purview=['A','B','C']
result=partitions(mechanism,purview,show=True)

(  / A ) X ( AC / BC )
(  / B ) X ( AC / AC )
(  / C ) X ( AC / AB )
( A /  ) X ( C / ABC )
( A / A ) X ( C / BC )
( A / B ) X ( C / AC )
( A / C ) X ( C / AB )
( C /  ) X ( A / ABC )
( C / A ) X ( A / BC )
( C / B ) X ( A / AC )
( C / C ) X ( A / AB )
( AC /  ) X (  / ABC )
( AC / A ) X (  / BC )
( AC / B ) X (  / AC )
( AC / C ) X (  / AB )


### Note:

Notice the results replicate the technical report and further than that, I CAUGHT TWO ERRORS (in red)

### Example 3:

Let's calculate MIPs and small phis:

In [233]:
mechanism=['A','B','C']
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
TPM=tpm(2,3,[0,1])
mec_pur_part=partitions(mechanism,purview, False)
ans1=mip(mechanism, purview, mec_pur_part[0], mec_pur_part[1],state,TPM,'cause', show=True)
print('\r\n')
ans2=mip(mechanism, purview, mec_pur_part[0], mec_pur_part[1],state,TPM,'effect', show=True)

[] ['A'] X ['A', 'B', 'C'] ['B', 'C'] 0.5
[] ['B'] X ['A', 'B', 'C'] ['A', 'C'] 0.5
[] ['C'] X ['A', 'B', 'C'] ['A', 'B'] 0.5
['A'] [] X ['B', 'C'] ['A', 'B', 'C'] 0.33333333333333337
['A'] ['A'] X ['B', 'C'] ['B', 'C'] 0.6666666666666667
['A'] ['B'] X ['B', 'C'] ['A', 'C'] 0.6666666666666667
['A'] ['C'] X ['B', 'C'] ['A', 'B'] 0.33333333333333337
['B'] [] X ['A', 'C'] ['A', 'B', 'C'] 0.33333333333333337
['B'] ['A'] X ['A', 'C'] ['B', 'C'] 0.6666666666666667
['B'] ['B'] X ['A', 'C'] ['A', 'C'] 0.6666666666666667
['B'] ['C'] X ['A', 'C'] ['A', 'B'] 0.33333333333333337
['C'] [] X ['A', 'B'] ['A', 'B', 'C'] 0.5
['C'] ['A'] X ['A', 'B'] ['B', 'C'] 0.5
['C'] ['B'] X ['A', 'B'] ['A', 'C'] 0.5
['C'] ['C'] X ['A', 'B'] ['A', 'B'] 0.5
['A', 'B'] [] X ['C'] ['A', 'B', 'C'] 0.5
['A', 'B'] ['A'] X ['C'] ['B', 'C'] 0.75
['A', 'B'] ['B'] X ['C'] ['A', 'C'] 0.75
['A', 'B'] ['C'] X ['C'] ['A', 'B'] 0.5
['A', 'C'] [] X ['B'] ['A', 'B', 'C'] 0.6666666666666667
['A', 'C'] ['A'] X ['B'] ['B', 'C'] 0.66666

## ...end of examples (integrated information  / MPI and φ “small-phi”).

## EXAMPLES (irreducibility).

### Example 1:

Let's find the Maximally irreducible cause of BC:

In [234]:
mechanism=['A','B','C']
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
TPM=tpm(2,3,[0,1])
    
ans=mice(mechanism,purview,state,TPM,'cause',show=True )

Mechanism: ['A', 'B', 'C'] possible purview: ['A']
MIP cause =  [] ['A'] X ['A', 'B', 'C'] [] -  MIPφ cause = 0.0 

Mechanism: ['A', 'B', 'C'] possible purview: ['B']
MIP cause =  [] ['B'] X ['A', 'B', 'C'] [] -  MIPφ cause = 0.0 

Mechanism: ['A', 'B', 'C'] possible purview: ['C']
MIP cause =  [] ['C'] X ['A', 'B', 'C'] [] -  MIPφ cause = 0.0 

Mechanism: ['A', 'B', 'C'] possible purview: ['A', 'B']
MIP cause =  ['A', 'B'] [] X ['C'] ['A', 'B'] -  MIPφ cause = 0.0 

Mechanism: ['A', 'B', 'C'] possible purview: ['A', 'C']
MIP cause =  ['C'] [] X ['A', 'B'] ['A', 'C'] -  MIPφ cause = 0.25 

Mechanism: ['A', 'B', 'C'] possible purview: ['B', 'C']
MIP cause =  ['C'] [] X ['A', 'B'] ['B', 'C'] -  MIPφ cause = 0.25 

Mechanism: ['A', 'B', 'C'] possible purview: ['A', 'B', 'C']
MIP cause =  ['A'] [] X ['B', 'C'] ['A', 'B', 'C'] -  MIPφ cause = 0.33333333333333337 

>>Maximally irreducible cause = pueview: ['A', 'B', 'C'] mip: ['A'] [] X ['B', 'C'] ['A', 'B', 'C'] 0.33333333333333337 



Now, if needed the same example above can be reproduced showning the partitions of each pair (mechanism-possible purview), just to see why each MIP cause was selected as such (partition with the smallest phi).

What's it takes is to go to the function mice and modify inside it the parameter show=True, every time this function calls mip.

Let's see how it looks:

In [159]:
mechanism=['A','C']
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
TPM=tpm(2,3,[0,1])
    
ans=mice(mechanism,purview,state,TPM,'cause',show=True )

Mechanism: ['A', 'C'] possible purview: ['A']
[] ['A'] X ['A', 'C'] [] 0.16666666666666666
['A'] [] X ['C'] ['A'] 0.16666666666666666
['A'] ['A'] X ['C'] [] 0.16666666666666666
['C'] [] X ['A'] ['A'] 0.16666666666666666
['C'] ['A'] X ['A'] [] 0.16666666666666666
['A', 'C'] [] X [] ['A'] 0.16666666666666666
MIP cause =  [] ['A'] X ['A', 'C'] [] -  MIPφ cause = 0.16666666666666666
MIP cause =  [] ['A'] X ['A', 'C'] [] -  MIPφ cause = 0.16666666666666666 

Mechanism: ['A', 'C'] possible purview: ['B']
[] ['B'] X ['A', 'C'] [] 0.16666666666666666
['A'] [] X ['C'] ['B'] 0.16666666666666666
['A'] ['B'] X ['C'] [] 0.0
['C'] [] X ['A'] ['B'] 0.0
['C'] ['B'] X ['A'] [] 0.16666666666666666
['A', 'C'] [] X [] ['B'] 0.16666666666666666
MIP cause =  ['A'] ['B'] X ['C'] [] -  MIPφ cause = 0.0
MIP cause =  ['A'] ['B'] X ['C'] [] -  MIPφ cause = 0.0 

Mechanism: ['A', 'C'] possible purview: ['C']
[] ['C'] X ['A', 'C'] [] 0.16666666666666666
['A'] [] X ['C'] ['C'] 0.16666666666666666
['A'] ['C'] X ['C'

### Example 2:

Let's find the concept of mechanism ABC:

In [235]:
mechanism=['A','B','C']
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
TPM=tpm(2,3,[0,1])

concept(mechanism,purview,state,TPM)

The concept of the mechanism ['A', 'B', 'C'] is:
cause: ['A', 'B', 'C'] / ['A', 'B', 'C'] - 0.33333333333333337
effect: ['A', 'B', 'C'] / ['A', 'B', 'C'] - 0.25 



([0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 0.25)

If, inside the function concept the parameter show is set to True when the fucntion mice is called. The result would look more robust:

In [37]:
mechanism=['A','B','C']
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
TPM=tpm(2,3,[0,1])

concept(mechanism,purview,state,TPM)

The concept of the mechanism ['A', 'B', 'C'] is:
cause: ['A', 'B', 'C'] / ['A', 'B', 'C'] - 0.08333333333333334
effect: ['A', 'B', 'C'] / ['A', 'B', 'C'] - 0.0625 



### Example 3:

Let's find the cause-effect structure of ABC

In [236]:
mechanism=['A','B','C']
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
TPM=tpm(2,3,[0,1])

ans=ces(mechanism,purview,state,TPM)

The concept of the mechanism ['A'] is:
cause: ['A'] / ['B', 'C'] - 0.16666666666666666
effect: ['A'] / ['B'] - 0.25 

The concept of the mechanism ['B'] is:
cause: ['B'] / ['A', 'C'] - 0.16666666666666666
effect: ['B'] / ['A'] - 0.25 

The concept of the mechanism ['C'] is:
cause: ['C'] / ['A', 'B'] - 0.5
effect: ['C'] / ['A', 'B'] - 0.25 

The concept of the mechanism ['A', 'B'] is:
cause: ['A', 'B'] / ['A', 'B', 'C'] - 0.25
effect: ['A', 'B'] / ['C'] - 0.375 

The concept of the mechanism ['A', 'C'] is:
cause: ['A', 'C'] / ['A', 'B', 'C'] - 0.16666666666666666
effect: ['A', 'C'] / ['A', 'B', 'C'] - 0.0 

The concept of the mechanism ['B', 'C'] is:
cause: ['B', 'C'] / ['A', 'B', 'C'] - 0.16666666666666666
effect: ['B', 'C'] / ['A', 'B'] - 0.25 

The concept of the mechanism ['A', 'B', 'C'] is:
cause: ['A', 'B', 'C'] / ['A', 'B', 'C'] - 0.33333333333333337
effect: ['A', 'B', 'C'] / ['A', 'B', 'C'] - 0.25 



Notice that even though the numbers diverge from those in the tech report (due to the difference between emd). The result is the same:

The only concept that has to be ruled out is AC whose small phi is equal to 0. So, the cause-effect structure is given by the concepts of nonzero small phi: A, B, C, AB, BC, ABC. Exactly like the tech report:

![title](fig3.png)



### Example 4:

Let's find the TPM for a cut system A --> BC:

In [237]:
TPM=tpm(2,3,[0,1])

tpm_cut(2,TPM,['A','B','C'],['A'],['B','C'],show=True)

TPM for element  A : 
 A      0    1
C B          
0 0  1.0  0.0
  1  0.0  1.0
1 0  0.0  1.0
  1  0.0  1.0 

TPM for element  B : 
 B    0    1
C          
0  1.0  0.0
1  0.5  0.5 

TPM for element  C : 
 C    0    1
B          
0  0.5  0.5
1  0.5  0.5 



,,A,0,1,0,1,0,1,0,1
,,B,0,0,1,1,0,0,1,1
,,C,0,0,0,0,1,1,1,1
A,B,C,,,,,,,,
0,0,0,0.5,0.00,0.0,0.00,0.5,0.00,0.0,0.00
1,0,0,0.5,0.00,0.0,0.00,0.5,0.00,0.0,0.00
0,1,0,0.0,0.50,0.0,0.00,0.0,0.50,0.0,0.00
1,1,0,0.0,0.50,0.0,0.00,0.0,0.50,0.0,0.00
0,0,1,0.0,0.25,0.0,0.25,0.0,0.25,0.0,0.25
1,0,1,0.0,0.25,0.0,0.25,0.0,0.25,0.0,0.25
0,1,1,0.0,0.25,0.0,0.25,0.0,0.25,0.0,0.25


See it aligns with all the results ine the technical report:

![title](fig4.png)

### Example 5:

Let's find the diference (big Phi) between the ce-structure of the whole ABC system and the cut A --> BC:

In [238]:
mechanism=['A','B','C']
purview=['A','B','C']
state={'A':1,'B':0,'C':0 }
TPM=tpm(2,3,[0,1])

print ('Concepts system:\r\n')
struct1=ces(mechanism,purview,state,TPM)
TPM2=tpm_cut(2,TPM,['A','B','C'],['A'],['B','C'],show=False)
print ('Concepts cut system:\r\n')
struct2=ces(mechanism,purview,state,TPM2)

calculate_phi(struct1, struct2)

Concepts system:

The concept of the mechanism ['A'] is:
cause: ['A'] / ['B', 'C'] - 0.16666666666666666
effect: ['A'] / ['B'] - 0.25 

The concept of the mechanism ['B'] is:
cause: ['B'] / ['A', 'C'] - 0.16666666666666666
effect: ['B'] / ['A'] - 0.25 

The concept of the mechanism ['C'] is:
cause: ['C'] / ['A', 'B'] - 0.5
effect: ['C'] / ['A', 'B'] - 0.25 

The concept of the mechanism ['A', 'B'] is:
cause: ['A', 'B'] / ['A', 'B', 'C'] - 0.25
effect: ['A', 'B'] / ['C'] - 0.375 

The concept of the mechanism ['A', 'C'] is:
cause: ['A', 'C'] / ['A', 'B', 'C'] - 0.16666666666666666
effect: ['A', 'C'] / ['A', 'B', 'C'] - 0.0 

The concept of the mechanism ['B', 'C'] is:
cause: ['B', 'C'] / ['A', 'B', 'C'] - 0.16666666666666666
effect: ['B', 'C'] / ['A', 'B'] - 0.25 

The concept of the mechanism ['A', 'B', 'C'] is:
cause: ['A', 'B', 'C'] / ['A', 'B', 'C'] - 0.33333333333333337
effect: ['A', 'B', 'C'] / ['A', 'B', 'C'] - 0.25 

Concepts cut system:

The concept of the mechanism ['A'] is:
c

1.0772569444444444